# To get the data from MoveBank API


![MoveBank Logo](https://www.movebank.org/cms/img/logo-movebank.png)


To use this API, you need an account in Movebank, register in (https://www.movebank.org/), this code is based on https://github.com/movebank/movebank-api-doc/blob/master/mb_Meschenmoser.py 

Once you get the data for the study you are interested in, you can store into ./Data folder in MagGeo, to easy load it into the Sequential or Parallel notebooks.

For more information about how to properly request the data using the MoveBank API, please visit (https://github.com/movebank/movebank-api-doc/blob/master/movebank-api.md). It is better for you to get familiar first with the way the data is requested, what attributes are avaliable and more importantly if the study you are looking for is open accesisable ( No username or pass is requiered) or you need to add your Movebank credentials to get access. You might also need to check what the API is returning checking the Individuals, Events and Studies URL requests.

The study and the data we are going to use in this example, is the following:

>Nolet BA, Kölzsch A, Elderenbosch M, van Noordwijk AJ (2016) Scaring waterfowl as a management tool: how much more do geese forage after disturbance? Journal of Applied Ecology. doi:10.1111/1365-2664.12698

In [7]:
import requests
import os
import hashlib
import csv
import json
import io
from datetime import datetime, timedelta
from pandas import json_normalize
import pandas as pd

import geopandas as gpd
import movingpandas as mpd
from shapely.geometry import Point
import hvplot.pandas  # noqa

In [8]:
import getpass
movebank_username = getpass.getpass()

In [9]:
movebank_password = getpass.getpass()

In [10]:
def callMovebankAPI(params):
    # Requests Movebank API with ((param1, value1), (param2, value2),).
    # Assumes the environment variables 'mbus' (Movebank user name) and 'mbpw' (Movebank password).
    # Returns the API response as plain text.

    response = requests.get('https://www.movebank.org/movebank/service/direct-read', params=params, auth=(movebank_username, movebank_password))
    print("Request " + response.url)
    if response.status_code == 200:  # successful request
        if 'License Terms:' in str(response.content):
            # only the license terms are returned, hash and append them in a subsequent request.
            # See also
            # https://github.com/movebank/movebank-api-doc/blob/master/movebank-api.md#read-and-accept-license-terms-using-curl
            print("Has license terms")
            hash = hashlib.md5(response.content).hexdigest()
            params = params + (('license-md5', hash),)
            # also attach previous cookie:
            response = requests.get('https://www.movebank.org/movebank/service/direct-read', params=params,
                                    cookies=response.cookies, auth=(movebank_username, movebank_password))
            if response.status_code == 403:  # incorrect hash
                print("Incorrect hash")
                return ''
        return response.content.decode('utf-8')
    print(str(response.content))
    return ''


def getStudies():
    studies = callMovebankAPI((('entity_type', 'study'), ('i_can_see_data', 'true'), ('there_are_data_which_i_cannot_see', 'false')))
    if len(studies) > 0:
        # parse raw text to dicts
        studies = csv.DictReader(io.StringIO(studies), delimiter=',')
        return [s for s in studies if s['i_can_see_data'] == 'true' and s['there_are_data_which_i_cannot_see'] == 'false']
    return []


def getStudiesBySensor(studies, sensorname='GPS'):
    return [s for s in studies if sensorname in s['sensor_type_ids']]


def getIndividualsByStudy(study_id):
    individuals = callMovebankAPI((('entity_type', 'individual'), ('study_id', study_id)))
    if len(individuals) > 0:
        return list(csv.DictReader(io.StringIO(individuals), delimiter=','))
    return []


def prettyPrint(l):
    print(json.dumps(l, indent=2))

In [11]:
allstudies = getStudies()

Request https://www.movebank.org/movebank/service/direct-read?entity_type=study&i_can_see_data=true&there_are_data_which_i_cannot_see=false


In [12]:
gpsstudies = getStudiesBySensor(allstudies, 'GPS')
prettyPrint(gpsstudies)

[
  {
    "acknowledgements": "Work by Michael Kaatz and local collaborators",
    "citation": "Experimental Great Bustard study",
    "go_public_date": "",
    "grants_used": "Funds from MPIAB covering the GSM tag.",
    "has_quota": "true",
    "i_am_owner": "false",
    "id": "7367444",
    "is_test": "false",
    "license_terms": "Please do not use data without contacting the PIs.",
    "license_type": "CUSTOM",
    "main_location_lat": "52.18",
    "main_location_long": "12.685",
    "name": "E-obs GSM Great Bustard",
    "number_of_deployments": "1",
    "number_of_individuals": "1",
    "number_of_tags": "1",
    "principal_investigator_address": "",
    "principal_investigator_email": "",
    "principal_investigator_name": "FV Gro\u00dftrappenschutz e.V. (Henrik Watzke)",
    "study_objective": "Experimentally tracking great bustard in Germany.",
    "study_type": "research",
    "suspend_license_terms": "false",
    "i_can_see_data": "true",
    "there_are_data_which_i_cannot_

In [13]:
individuals = getIndividualsByStudy(study_id=133992043) 
# Using the readme of https://www.datarepository.movebank.org/handle/10255/move.546 you can get the study ID

prettyPrint(individuals)

Request https://www.movebank.org/movebank/service/direct-read?entity_type=individual&study_id=133992043
[
  {
    "comments": "",
    "death_comments": "",
    "earliest_date_born": "",
    "exact_date_of_birth": "",
    "id": "138936193",
    "latest_date_born": "",
    "local_identifier": "2704",
    "nick_name": "",
    "ring_id": "",
    "sex": "m",
    "taxon_canonical_name": "Anser albifrons",
    "timestamp_start": "2013-09-30 08:30:48.000",
    "timestamp_end": "2014-04-01 18:00:56.000",
    "number_of_events": "706",
    "number_of_deployments": "1",
    "sensor_type_ids": "GPS",
    "taxon_detail": ""
  },
  {
    "comments": "",
    "death_comments": "",
    "earliest_date_born": "",
    "exact_date_of_birth": "",
    "id": "138936196",
    "latest_date_born": "",
    "local_identifier": "2711",
    "nick_name": "",
    "ring_id": "",
    "sex": "m",
    "taxon_canonical_name": "Anser albifrons",
    "timestamp_start": "2013-08-29 13:30:44.000",
    "timestamp_end": "2013-10

In [14]:
STUDY_ID = 133992043
INDIVIDUAL_ID = '138936193'

In [15]:
params = (('entity_type', 'event'), ('study_id', STUDY_ID), ('individual_id', INDIVIDUAL_ID), ('sensor_type_id', 653))
events_csv = callMovebankAPI(params)

Request https://www.movebank.org/movebank/service/direct-read?entity_type=event&study_id=133992043&individual_id=138936193&sensor_type_id=653


In [16]:
df = pd.read_csv(io.StringIO(events_csv))
df

,timestamp,location_lat,location_long,individual_id,tag_id
0,2013-09-30 08:30:48.000,62.707029,36.132815,138936193,138936194
1,2013-09-30 13:31:26.000,60.148437,32.238147,138936193,138936194
2,2013-10-02 08:30:43.000,54.331167,12.581129,138936193,138936194
3,2014-03-17 22:00:13.000,52.108574,6.217192,138936193,138936194
4,2014-03-17 22:30:28.000,52.194110,6.505170,138936193,138936194
...,...,...,...,...,...
701,2014-04-01 13:31:22.000,52.326469,39.370952,138936193,138936194
702,2014-04-01 14:01:06.000,52.326355,39.370578,138936193,138936194
703,2014-04-01 14:31:21.000,52.326355,39.370580,138936193,138936194
704,2014-04-01 15:01:16.000,52.326334,39.370876,138936193,138936194


In [17]:
export_csv = df.to_csv (r"../data/MoveBank.csv", index = None, header=True)
# Save the Movebank data from the API, to the Data folder in MagGeo

In [18]:
emp_df = pd.read_csv(r"../data/MoveBank.csv")

## Let's Map the data we just got from MoveBank API

To map the data we can use geopandas (https://geopandas.org/), Hvplot(https://hvplot.holoviz.org/) and moving pandas (https://anitagraser.github.io/movingpandas/) to get a nice and interactive tool to explore the data we are looking at.

In [19]:
df.dropna(subset=['location_long', 'location_lat'], inplace=True)

In [20]:
gdf = gpd.GeoDataFrame(
    df.drop(['location_long', 'location_lat'], axis=1),
    crs='epsg:4326',
    geometry=[Point(xy) for xy in zip(df.location_long, df.location_lat)])

In [21]:
gdf

,timestamp,individual_id,tag_id,geometry
0,2013-09-30 08:30:48.000,138936193,138936194,POINT (36.13282 62.70703)
1,2013-09-30 13:31:26.000,138936193,138936194,POINT (32.23815 60.14844)
2,2013-10-02 08:30:43.000,138936193,138936194,POINT (12.58113 54.33117)
3,2014-03-17 22:00:13.000,138936193,138936194,POINT (6.21719 52.10857)
4,2014-03-17 22:30:28.000,138936193,138936194,POINT (6.50517 52.19411)
...,...,...,...,...
701,2014-04-01 13:31:22.000,138936193,138936194,POINT (39.37095 52.32647)
702,2014-04-01 14:01:06.000,138936193,138936194,POINT (39.37058 52.32636)
703,2014-04-01 14:31:21.000,138936193,138936194,POINT (39.37058 52.32635)
704,2014-04-01 15:01:16.000,138936193,138936194,POINT (39.37088 52.32633)


In [22]:
gdf.hvplot(title=f'Movebank events of individual {INDIVIDUAL_ID}',
           geo=True,
           tiles='CartoLight',
           frame_width=700,
           frame_height=500)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Longitude,Latitude]

### Using Moving Pandas we could map the trajectory based on the GPS points.

In [23]:
gdf['timestamp'] = pd.to_datetime(gdf['timestamp'])
gdf.set_index('timestamp', inplace=True)

In [24]:
traj = mpd.Trajectory(gdf, 'tag_id')

In [25]:
traj.hvplot(title=f'Trajectory of individual {INDIVIDUAL_ID}',
            c='tag_id',
            frame_width=700,
            frame_height=500,
            line_width=7.0,
            tiles='CartoLight',
            cmap='Viridis',
            colorbar=True)

:Overlay
   .Tiles.I :Tiles   [x,y]
   .Path.I  :Path   [Longitude,Latitude]   (tag_id)

Of course if you get more attributes from the MoveBack API request you migth plot differente attrubutes, but the process is similar from what we've done. 